In [1]:
#### 패키지 임포트
import pandas as pd
import numpy as np
import re
from tqdm.notebook import tqdm
import os
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
pd.set_option('display.max_columns', None)

In [2]:
topwords50 = ['지역', '산림', '문화', '숲', '사람', '교육', '자연', '마을', '이용', '산', '나무', '관광', '생활', '역사', '마음', '활용', '이야기', '미래', '가을', '예술', '청소년', '전통', '삶', '음식', '여름', '소나무', '봄', '이해', '여성', '겨울', '지식', '공감', '정신', '공동체', '사상', '성인', '공예', '정서', '창조', '고급', '남성', '신앙', '교양', '유람', '유람기', '향유', '의식주', '포용', '금송', '성황당', '서낭당', '친화력']
# '학습'을 제거하고 '교육'으로 통합시킴

In [3]:
data = pd.read_csv('산림문화_본문to형태소.csv')
print(data.shape, data.columns)
docs = data['morphs'].to_list()
print(len(docs))

(20216, 1) Index(['morphs'], dtype='object')
20216


In [4]:
#### 각각의 본문이 str타입('나는 어제 밥을 먹었다')일 경우 list(['나', '는', '어제', '밥', '을' '먹다', '이다'])로 바꿔줌
docs2 = docs.copy()
for i in tqdm(range(len(docs))):
    filtered_lst = []
    for w in docs2[i].split():
        if w in topwords50:
            filtered_lst.append(w)
        elif w == '학습':
            filtered_lst.append('교육')
    else:
        docs2[i] = " ".join(filtered_lst)

  0%|          | 0/20216 [00:00<?, ?it/s]

In [ ]:
# #### 중복 제거 잘 됐나 개수로 확인
# print(len(docs[0]))
# print(len(docs2[0]))
# print(docs2[0][0:10])

In [5]:
# TF-IDF 생성
""" CountVectorizer로 TF-IDF를 만들어서 변환하는 방법도 있지만 """
""" 처음부터 Tfidf를 구해주는 함수(TfidfVectorizer)가 있기 때문에 이를 사용할 것임. """

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

# 데이터 프레임에서 분석할 문서를 가져와서 쓸 경우 꼭 리스트로 변환해서 담아줘야 함.
corpus = docs2.copy()

In [6]:
# stop_words에는 위에서 조사만 추출해서 만든 불용어 리스트 사용
tfidf = TfidfVectorizer(token_pattern=r'\w{1,}')
# tfidf_v = TfidfVectorizer(stop_words = hyeong_josa2).fit(corpus)
tdm = tfidf.fit_transform(corpus)
print(tdm.shape)

(20216, 50)


In [7]:
# 만든 TF-IDF를 데이터 프레임에 담기 위해 우선 numpy.array()로 변환함
tfidf_array = tdm.toarray()
tfidf_array

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.71800414],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.39288936, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [8]:
# 변환한 array를 다시 pd.Dataframe으로 바꿈
tfidf_DF = pd.DataFrame(tfidf_array)
tfidf_DF

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.572309,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.178263,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.800428,0.0,0.000000,0.0,0.0,0.0,0.000000
1,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.696039,0.0,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.718004
2,0.000000,0.000000,0.0,0.382963,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.280126,0.000000,0.0,0.35725,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.673695,0.0,0.000000,0.000000,0.0,0.382689,0.0,0.0,0.000000,0.000000,0.216625,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
3,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.337822,0.00000,0.291865,0.0,0.000000,0.0,0.0,0.000000,0.494388,0.0,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.339383,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.450623,0.261306,0.0,0.295866,0.0,0.231109,0.000000,0.0,0.000000,0.0,0.0,0.0,0.169706
4,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,1.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20211,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.11969,0.000000,0.0,0.573447,0.0,0.0,0.000000,0.026061,0.0,0.00000,0.028980,0.0,0.000000,0.000000,0.000000,0.017345,0.0,0.00000,0.0,0.730221,0.0,0.000000,0.0,0.214686,0.000000,0.0,0.000000,0.0,0.0,0.199669,0.000000,0.151521,0.0,0.000000,0.0,0.000000,0.089654,0.0,0.069541,0.0,0.0,0.0,0.000000
20212,0.000000,0.015817,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.994147,0.0,0.0,0.000000,0.011536,0.0,0.00000,0.019241,0.0,0.000000,0.046105,0.028524,0.023032,0.0,0.00000,0.0,0.083912,0.0,0.014221,0.0,0.000000,0.014294,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
20213,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
20214,0.392889,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.00000,0.266634,0.0,0.561708,0.159728,0.000000,0.000000,0.0,0.50157,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.426548,0.0,0.0,0.0,0.000000


In [9]:
# 이 데이터프레임의 컬럼명은 숫자로 되어 있음. 이를 feature_names로 바꿔줘야 함.
featurenames = tfidf.get_feature_names_out()
tfidf_DF.columns = featurenames

In [10]:
tfidf_DF

,가을,겨울,고급,공감,공동체,공예,관광,교양,교육,금송,나무,남성,마을,마음,문화,미래,봄,사람,사상,산,산림,삶,생활,서낭당,성인,소나무,숲,신앙,여름,여성,역사,예술,유람,음식,의식주,이야기,이용,이해,자연,전통,정서,정신,지식,지역,창조,청소년,친화력,포용,향유,활용
0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.572309,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.178263,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.800428,0.0,0.000000,0.0,0.0,0.0,0.000000
1,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.696039,0.0,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.718004
2,0.000000,0.000000,0.0,0.382963,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.280126,0.000000,0.0,0.35725,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.673695,0.0,0.000000,0.000000,0.0,0.382689,0.0,0.0,0.000000,0.000000,0.216625,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
3,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.337822,0.00000,0.291865,0.0,0.000000,0.0,0.0,0.000000,0.494388,0.0,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.339383,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.450623,0.261306,0.0,0.295866,0.0,0.231109,0.000000,0.0,0.000000,0.0,0.0,0.0,0.169706
4,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,1.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20211,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.11969,0.000000,0.0,0.573447,0.0,0.0,0.000000,0.026061,0.0,0.00000,0.028980,0.0,0.000000,0.000000,0.000000,0.017345,0.0,0.00000,0.0,0.730221,0.0,0.000000,0.0,0.214686,0.000000,0.0,0.000000,0.0,0.0,0.199669,0.000000,0.151521,0.0,0.000000,0.0,0.000000,0.089654,0.0,0.069541,0.0,0.0,0.0,0.000000
20212,0.000000,0.015817,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.994147,0.0,0.0,0.000000,0.011536,0.0,0.00000,0.019241,0.0,0.000000,0.046105,0.028524,0.023032,0.0,0.00000,0.0,0.083912,0.0,0.014221,0.0,0.000000,0.014294,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
20213,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
20214,0.392889,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.00000,0.266634,0.0,0.561708,0.159728,0.000000,0.000000,0.0,0.50157,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.426548,0.0,0.0,0.0,0.000000


In [ ]:
# 각 단어에 대한 TF-IDF의 합
word_TFIDF_sum = pd.DataFrame({
    '단어': tfidf.get_feature_names_out(),
    'tf-idf': tdm.sum(axis=0).flat
})
word_TFIDF_sum = word_TFIDF_sum.sort_values(by='tf-idf', ascending=False).reset_index(drop=True)
word_TFIDF_sum

In [ ]:
# 각 단어에 대한 TF-IDF의 합
word_TFIDF_average = pd.DataFrame({
    '단어': tfidf.get_feature_names_out(),
    'tf-idf': tdm.mean(axis=0).flat
})
word_TFIDF_average = word_TFIDF_average.sort_values(by='tf-idf', ascending=False).reset_index(drop=True)
word_TFIDF_average

In [15]:
word_TFIDF_average.to_excel("산림문화_tfidf_top50.xlsx", index=False)